In [1]:
# Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from matplotlib.colors import ListedColormap
from matplotlib import cm
from scipy import signal
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets
import time
import random
import mplcursors
from matplotlib.animation import FuncAnimation, PillowWriter
from numpy.random import rand
from scipy.stats import pearsonr
import ipywidgets as widgets

# Import the necessary functions from the libraries
from wolffdemo_funcs import *

%matplotlib widget 
# Makes all interactive plots possible, both on vscode and Jupyter notebooks

# %matplotlib qt 
# Restart the kernel and uncomment this line to show plots in a separate window instead

In [ ]:

class WolffDemo:
    def __init__(self, N=100):
        # Parameters
        self.N = N
        self.J = 1
        self.kT = 2/np.log(1 + np.sqrt(2)) * 1.0  # T_c
        self.p = 1 - np.exp(-2*self.J/self.kT)
        
        # Initialize grids with -1 and 1
        self.grid0 = np.sign(0.5 - rand(N, N))
        self.grid = self.grid0.copy()
        
        # Get adjacency information
        self.adj = myNeighbors(np.arange(N*N), N)
        
        # Initialize state tracking
        self.correlations = [pearsonr(self.grid.flatten(), self.grid0.flatten())[0]]
        self.state = 'initial'
        self.iteration = 0
        
        # Create widgets
        self.next_button = widgets.Button(description='Next Step')
        self.next_button.on_click(self.step)
        
        self.reset_button = widgets.Button(description='Reset')
        self.reset_button.on_click(self.reset)
        
        self.status_label = widgets.Label(value='Initial State')
        self.corr_label = widgets.Label(value=f'Correlation: {self.correlations[-1]:.3f}')
        
        # Create layout
        self.button_box = widgets.HBox([self.next_button, self.reset_button])
        self.info_box = widgets.VBox([self.status_label, self.corr_label])
        self.output = widgets.Output()
        
        self.container = widgets.VBox([
            self.info_box,
            self.output,
            self.button_box
        ])
        
        # Initial display
        self.grid_display = self.grid.copy()
        self.update_display()
        display(self.container)
        print(f'Iteratively add neighbor to cluster with probability {self.p:.3f}')

    def update_display(self):
        """Update the plot display"""
        with self.output:
            clear_output(wait=True)
            
            # Create figure with specified ratio
            fig = plt.figure(figsize=(8, 10))
            gs = plt.GridSpec(5, 1, height_ratios=[4, 4, 4, 4, 1], figure=fig)
            
            # Main plot
            ax1 = fig.add_subplot(gs[0:4])
            
            # Create custom colormap for the grid
            colors = ['white', 'black', '#E6550D', '#FEE6CE']
            custom_cmap = plt.cm.colors.ListedColormap(colors)
            
            # Display grid
            im = ax1.imshow(self.grid_display, cmap=custom_cmap, aspect='equal', vmin=0, vmax=4)
            cbar = plt.colorbar(im, ax=ax1, ticks=[0.5, 1.5, 2.5, 3.5], fraction=0.046, pad=0.04)
            cbar.set_ticklabels(['down', 'up', 'seed', 'cluster'])
            ax1.set_xticks([])
            ax1.set_yticks([])
            
            # Set title based on state
            if self.state == 'seed':
                ax1.set_title('SEED')
            elif self.state == 'cluster':
                ax1.set_title('CLUSTER')
            else:
                ax1.set_title(f'Grid State - Iteration {self.iteration}')
            
            # Correlation plot
            ax2 = fig.add_subplot(gs[4])
            ax2.plot(self.correlations, '.-k')
            ax2.set_xlabel('Iteration')
            ax2.set_ylabel('Corr0')
            ax2.set_ylim(-0.1, 1)
            ax2.set_title(f'{self.correlations[-1]:.3f}')
            
            plt.subplots_adjust(hspace=-0.25)  # Reduce space between plots
            
#             plt.tight_layout()
            display(fig)
            plt.close(fig)

    def step(self, b=None):
        """Advance the simulation one step"""
        if self.state == 'initial':
            # Get cluster update
            self.C, self.seed = WolffIteration(self.N, self.p, self.grid, self.adj)
            self.theNewSpin = -self.grid[np.unravel_index(self.seed, (self.N,self.N))]
            
            # Show seed
            self.grid_display = np.where(self.grid == -1, 0, 1)  # Convert -1 to 0 for display
            self.grid_display[np.unravel_index(self.seed, (self.N,self.N))] = 2
            self.status_label.value = 'Seed Selected'
            self.state = 'seed'
            
        elif self.state == 'seed':
            # Show cluster
            self.grid_display = np.where(self.grid == -1, 0, 1)  # Convert -1 to 0 for display
            for c in self.C:
                self.grid_display[np.unravel_index(c, (self.N,self.N))] = 3
            self.grid_display[np.unravel_index(self.seed, (self.N,self.N))] = 2
            self.status_label.value = 'Cluster Identified'
            self.state = 'cluster'
            
        elif self.state == 'cluster':
            # Update spins
            for c in self.C:
                self.grid[np.unravel_index(c, (self.N,self.N))] = self.theNewSpin
            self.grid_display = np.where(self.grid == -1, 0, 1)  # Convert -1 to 0 for display
            
            # Update correlation
            self.correlations.append(
                pearsonr(self.grid.flatten(), self.grid0.flatten())[0]
            )
            self.corr_label.value = f'Correlation: {self.correlations[-1]:.3f}'
            
            self.status_label.value = 'Updated State'
            self.state = 'initial'
            self.iteration += 1
            
        self.update_display()

    def reset(self, b=None):
        """Reset the simulation to initial state"""
        self.grid = self.grid0.copy()
        self.correlations = [pearsonr(self.grid.flatten(), self.grid0.flatten())[0]]
        self.state = 'initial'
        self.iteration = 0
        self.grid_display = np.where(self.grid == -1, 0, 1)  # Convert -1 to 0 for display
        self.status_label.value = 'Initial State'
        self.corr_label.value = f'Correlation: {self.correlations[-1]:.3f}'
        self.update_display()

# Create and display the demo
demo = WolffDemo()

Iteratively add neighbor to cluster with probability 0.586
